# 1. 상품데이터 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

url = 'https://www.ssfshop.com/public/goods/detail/listReview'

item_ids = [
    "GM0023102645990", "GR2Y24040463208", "GM0023102645988", "GM0023102645987", "GQ3621011525906",
    "GR2Y24040463238", "GM0022081981912", "GM0022081981911", "GQ3621011525909", "GM0024102287131",
    "GM0024102287130", "GR2Y24022710834", "GR5L24041844965", "GQ3623092161279", "GQ3621061670876",
    "GR2Y24090242142", "GR2Y24090242342", "GR5L24041842074", "GM0023092272987", "GM0023092272991",
    "GM0024082116564", "GM0023092272992", "GQ3623101885649", "GQ3622022185706", "GR3L24030545490",
    "GM0022110300679", "GR2Y24030654526", "GR1F24022604881", "GM0022101479641", "GR2Y24090242255"
]

review_list = []

for item_id in item_ids:
    print(f"Collecting reviews for ITEM_ID: {item_id}")
    page_no = 1
    collected_reviews = 0
    while collected_reviews < 10:
        params = {
            'pageNo': page_no,
            'sortFlag': '',
            'reviewSize': '',
            'reviewFlag': '',
            'godNo': item_id,
            'selectAllYn': 'N',
            'reviewColor': '',
            'reviewFilterHeight': '',
            'reviewFilterWeight': '',
            'reviewFilterUsldaySize': '',
            'reviewFilterPantsUsldaySize': '',
            'godEvlTurn': 0
        }

        response = requests.get(url, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')
        reviews = soup.find_all('li', {'data-godno': item_id})

        if not reviews:
            break

        print(f"  Collecting page {page_no}")

        for review in reviews:
            if collected_reviews >= 10:
                break

            # Extract rating
            stars = review.find('span', class_='rate')
            rating = int(re.search(r'point(\d+)', stars['class'][1]).group(1)) if stars and 'class' in stars.attrs else 'N/A'

            # Extract date
            date = review.find('span', class_='list-date')
            date = date.text.strip() if date else 'N/A'

            # Extract user
            user = review.find('span', class_='list-id')
            user = user.text.strip() if user else 'N/A'

            # Extract review text
            text = review.find('p', class_='review-txts')
            text = text.text.strip() if text else 'N/A'

            # Extract image
            image = review.find('img')['data-original'] if review.find('img') else 'No Image'

            review_list.append({
                'ITEM_ID': item_id,
                'User': user,
                'Date': date,
                'Rating': rating,
                'Review': text,
                'Image': image
            })

            collected_reviews += 1

        page_no += 1

# 데이터프레임 생성
df = pd.DataFrame(review_list)
df


# 2. 리뷰데이터 크롤링

In [ ]:
import re  # 정규식 사용
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 크롬 드라이버 설정
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

try:
    # 검색 페이지 열기
    search_url = "https://www.ssfshop.com/public/search/search/view?serviceType=SRC&keyword=%EC%9E%A0%EC%98%B7&orderView=SORT_EVL_CNT&pageNo=1&_csrf=98490275-b1f7-4dd9-b4ff-3d807122d66b&tab=GOD&schType=ALL"
    driver.get(search_url)
    time.sleep(3)  # 페이지 로딩 대기

    # 상위 10개 아이템 링크 수집
    item_links = []
    items = driver.find_elements(By.CSS_SELECTOR, "li.god-item > a")[:10]  # 상위 10개 아이템 선택
    for item in items:
        link = item.get_attribute("href")
        full_link = f"https://www.ssfshop.com{link}" if not link.startswith("http") else link  # 절대경로 처리
        item_links.append(full_link)

    # 리뷰 데이터 저장용 리스트
    reviews_data = {
        "상품코드": [],
        "리뷰일자": [],
        "평점": [],
        "리뷰내용": [],
        "아이템링크": []  # 아이템 링크 추가
    }

    # 각 아이템 페이지로 이동하여 리뷰 수집
    for link in item_links:
        driver.get(link)
        time.sleep(3)  # 페이지 로딩 대기

        # 최대 리뷰 수
        max_reviews = 50
        current_count = 0

        while current_count < max_reviews:
            # 리뷰 리스트 가져오기
            review_elements = driver.find_elements(By.CSS_SELECTOR, "li[data-godno]")
            for review_element in review_elements:
                if current_count >= max_reviews:
                    break
                try:
                    # 상품코드
                    product_code = review_element.get_attribute("data-godno")

                    # 리뷰일자
                    try:
                        review_date = review_element.find_element(By.CSS_SELECTOR, "span.list-date").get_attribute("innerText").strip()
                    except:
                        review_date = "N/A"

                    # 평점
                    try:
                        rate_class = review_element.find_element(By.CSS_SELECTOR, "span.rate").get_attribute("class").strip()
                        # 정규식을 사용하여 평점 숫자만 추출
                        rate = re.search(r'\d+', rate_class).group() if re.search(r'\d+', rate_class) else "N/A"
                    except:
                        rate = "N/A"

                    # 리뷰 내용
                    try:
                        review_text = review_element.find_element(By.CSS_SELECTOR, "p.review-txts").get_attribute("innerText").strip()
                    except:
                        review_text = "N/A"

                    # 데이터 저장
                    reviews_data["상품코드"].append(product_code)
                    reviews_data["리뷰일자"].append(review_date)
                    reviews_data["평점"].append(rate)
                    reviews_data["리뷰내용"].append(review_text)
                    reviews_data["아이템링크"].append(link)  # 현재 아이템 링크 저장

                    current_count += 1
                except Exception as e:
                    print(f"Error processing review: {e}")

            # 다음 페이지로 이동
            try:
                next_page = driver.find_element(By.CSS_SELECTOR, "a.next")
                next_page.click()
                time.sleep(3)  # 페이지 로딩 대기
            except:
                print("No more pages to navigate.")
                break

    # 데이터프레임 생성
    df = pd.DataFrame(reviews_data)

    # 저장 경로 설정
    output_path = r"/Users/Jooah_kim/주아리/2025/inner circle/reviews_data22.xlsx"

    # 데이터 저장
    df.to_excel(output_path, index=False)
    print(f"Data saved to {output_path}")

finally:
    # 드라이버 종료
    driver.quit()

## 텍스트 데이터 분석

<aside>
💡 텍스트 데이터 분석 프로세스

1. 데이터수집
    - 소셜 미디어, 리뷰, 뉴스, 대화 데이터 등 다양한 출처에서 텍스트 데이터를 수집.
2. 전처리
    - 불필요한 단어 제거(불용어 처리), 토큰화, 형태소 분석 등.
3. 분석 수행
    - 분석 목적에 따라 빈도 분석, 감정 분석, 네트워크 분석 등을 수행.
4. 시각화
    - 워드 클라우드, 네트워크 그래프, 감정 스코어 분포 등을 통해 결과를 효과적으로 전달.
</aside>

### 1. 빈도분석

- **개념**
    - 텍스트 데이터 내 특정 단어, 구문 또는 개체가 등장하는 빈도를 계산하여 주요 키워드 및 패턴을 분석하는 방법입니다.
- **활용 예시**
    - 고객 리뷰 데이터에서 가장 많이 언급되는 제품의 장점/단점을 파악.
    - 뉴스 기사에서 특정 이슈(예: "기후 변화", "AI 기술")의 언급 빈도를 분석하여 트렌드 확인.
- **주요 분석 지표**
    - 단어 빈도수 (Word Count)
    - TF-IDF (Term Frequency-Inverse Document Frequency)

### 2. 감정분석

- **개념**
    - 텍스트 데이터의 정서적 경향(긍정, 부정, 중립)을 파악하는 분석 기법입니다. 주로 자연어 처리(NLP) 기법과 사전 기반 또는 기계 학습 모델을 활용합니다.
- **활용 예시**
    - 소셜 미디어 게시물에서 브랜드에 대한 고객 감정 분석.
    - 영화 리뷰나 상품 리뷰에서 긍정/부정 평가의 비율 계산.
- **주요 분석 기법**
    - 감정 단어 사전 기반 분석 (e.g., NRC, VADER)
    - 딥러닝 모델 기반 감정 분류 (e.g., BERT, GPT 기반 모델)

### 3. ⭐️ 네트워크 분석 ⭐️

- **개념**
    - 텍스트 데이터에서 단어, 문장, 문서 간의 관계를 네트워크 형태로 모델링하여 분석하는 기법입니다.
        - 노드(Node): 단어, 구문, 개체 등
        - 엣지(Edge): 노드 간 연결 또는 연관성
- **분석 유형**
    - **연결강도**단어 또는 노드 간의 관계의 강도를 측정. 두 단어가 함께 나타나는 빈도수로 측정하거나, 상관 계수 기반으로 분석.
        - **활용 예시**
            - 고객 리뷰에서 "배터리"와 "지속시간"의 높은 연결강도를 확인하여 제품 개선에 활용.
            - 뉴스 데이터에서 특정 키워드 간의 연관성을 분석해 트렌드 도출.
    - **클러스터링**네트워크에서 서로 밀접히 연결된 노드 그룹을 파악.
        - **활용 예시**
            - 온라인 커뮤니티의 주제별 논의 클러스터 분류.
            - 논문 키워드 네트워크에서 연구 주제 도출

### 4. 노드 분석

- **개념**
    - 네트워크에서 개별 노드의 특성을 분석하여 중요도, 중심성 등을 파악하는 기법입니다.
- **주요 분석 지표**
    - **중심성(Centrality)**네트워크 내에서 특정 노드의 중요성을 나타냄.
        - **Degree Centrality**: 노드와 직접 연결된 엣지의 개수.
        - **Betweenness Centrality**: 노드가 다른 노드 간 최단 경로에 포함되는 정도.
        - **Closeness Centrality**: 노드가 다른 모든 노드와 얼마나 가까운가를 측정.
    - **PageRank**구글 검색 알고리즘의 핵심으로, 연결 구조에서 노드의 중요도를 평가.
- **활용 예시**
    - 논문 네트워크에서 가장 많이 인용된 논문 파악.
    - 소셜 미디어 데이터에서 인플루언서 또는 중심 키워드 도출